In [66]:
import pandas as pd
import numpy as np
import z3
from tqdm import tqdm

In [67]:
macrotick = 100
sync_error = 0
time_out = 45 * 60

In [68]:
NUM_FLOW = 10
DATA_NAME = "harmonic0"
TOPO_NAME = "0"

task = pd.read_csv("../../data/utilization/utilization_5_9_task.csv")
network = pd.read_csv("../../data/utilization/utilization_5_9_topo.csv")
# task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:NUM_FLOW]
# network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

In [69]:
U = 3

In [70]:
min_size = 1

## 1. Model

In [71]:
s = z3.Solver()
s.set("timeout", time_out * 1000)

In [72]:
net_var = {}
for _, row in network.iterrows():
    net_var.setdefault(row['link'], {})
    net_var[row['link']]['s'] = row['rate']
    net_var[row['link']]['d'] = row['t_proc']
    net_var[row['link']]['c'] = row['q_num']
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

In [73]:
task_var = {}

In [74]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [75]:
## Assume task is strictly periodic
for i, row in task.iterrows(): 
    task_var.setdefault(i, {})
    route = eval(str(next(bfs_paths(net, int(row['src']), int(eval(row['dst'])[0])))))
    for _i, a in enumerate(route[:-1]):
        link = str((a, route[_i + 1]))
        task_var[i].setdefault(link, {})

        # task_var[i][link]['r'] = z3.IntVector("r_" + str(i) + '_' + str(link), int(LCM/row['period']))
        # task_var[i][link]['f'] = z3.IntVector("r_" + str(i) + '_' + str(link), int(LCM/row['period']))
        task_var[i][link]['N'] = z3.BoolVector("n_" + str(i) + '_' + str(link), int(LCM/row['period']))
        task_var[i][link]['T'] = row['period']
        task_var[i][link]['L'] = int(row['size'] * 8 / net_var[str(link)]['s'])
        for j in range(0, int(LCM/row['period'])):
            task_var[i][link].setdefault(j, {})
            task_var[i][link][j]['r'] = z3.IntVector("r_" + str(i) + str(j) + '_' + str(link), U)
            task_var[i][link][j]['f'] = z3.IntVector("f_" + str(i) + str(j) + '_' + str(link), U)
            s.add(
                j * row['period'] <= task_var[i][link][j]['r'][0],
                task_var[i][link][j]['f'][U - 1] <= (j + 1) * row['period']
            )

## 2. Constraints

In [76]:
for i, k in [(i, k) for i in task_var for k in task_var if i != k]:
    for link in [link for link in net_var if link in task_var[i] and link in task_var[k]]:
        for j, l in [(j, l) for j in range(int(LCM / task_var[i][link]['T'])) 
        for l in range(int(LCM / task_var[k][link]['T']))]:
            s.add(
                z3.Implies(
                    z3.Or(
                        z3.And(task_var[i][link]['N'][j] == True, task_var[k][link]['N'][l] == True),
                        z3.And(task_var[i][link]['N'][j] == False, task_var[k][link]['N'][l] == False)
                    ),
                    z3.Or(
                        task_var[i][link][j]['r'][0] >= task_var[k][link][l]['f'][-1],
                        task_var[k][link][l]['r'][0] >= task_var[i][link][j]['f'][-1]
                        # task_var[i][link][j]['r'][0] - task_var[k][link][l]['r'][0] >= task_var[k][link]['L'],
                        # task_var[k][link][l]['r'][0] - task_var[i][link][j]['r'][0] >= task_var[i][link]['L'],
                    )
                )
            )

In [77]:
for i, k in tqdm([(i, k) for i in task_var for k in task_var if i != k]):
    for link in [link for link in net_var if link in task_var[i] and link in task_var[k]]:
        for j, l in [(j, l) for j in range(int(LCM / task_var[i][link]['T'])) 
        for l in range(int(LCM / task_var[k][link]['T']))]:
            temp = []
            for y, z in [(y, z) for y in range(0, U - 1) for z in set(range(0, U - 1)) - set([y, y+1])]:
                temp.append(
                        z3.And(
                            z3.Or(
                            task_var[i][link][j]['f'][0] <= task_var[k][link][l]['r'][0],
                            task_var[k][link][l]['f'][0] <= task_var[i][link][j]['r'][0]
                            ),
                            task_var[k][link][l]['f'][y] <= task_var[i][link][j]['r'][0],
                            task_var[k][link][l]['r'][y + 1] >= task_var[i][link][j]['f'][0],
                            task_var[i][link][j]['f'][0] - task_var[i][link][j]['r'][0] == task_var[i][link]['L'],
                        )

                        # z3.Or(
                        #     task_var[i][link][j]['f'][0] <= task_var[k][link][l]['r'][0],
                        #     task_var[k][link][l]['f'][0] <= task_var[i][link][j]['r'][0],
                        #     z3.And(
                        #         task_var[k][link][l]['f'][y] - task_var[k][link][l]['r'][y] >= min_size,
                        #         task_var[k][link][l]['f'][y + 1] - task_var[k][link][l]['r'][y + 1] >= min_size,
                        #         z3.Or(
                        #             task_var[k][link][l]['f'][z] - task_var[k][link][l]['r'][z] == 0,
                        #             task_var[k][link][l]['f'][z] - task_var[k][link][l]['r'][z] >= min_size
                        #         ),
                        #         z3.Or(
                        #             task_var[k][link][l]['f'][y] == task_var[i][link][j]['r'][0],
                        #             task_var[k][link][l]['f'][y] + min_size <= task_var[i][link][j]['r'][0]
                        #         ),
                        #         z3.Or(
                        #             task_var[k][link][l]['r'][y + 1] == task_var[i][link][j]['f'][0],
                        #             task_var[k][link][l]['r'][y + 1] >= task_var[i][link][j]['f'][0] + min_size,
                        #         ),
                        #         task_var[i][link][j]['f'][0] - task_var[i][link][j]['r'][0] == task_var[i][link]['L'],   
                        #     )
                        # )
                )

            s.add(
                z3.Implies(
                    z3.And(task_var[i][link]['N'][j] == True, task_var[k][link]['N'][l] == False),
                    z3.And(temp)
                ),
                z3.Implies(
                    z3.And(task_var[i][link]['N'][j] == True),
                    task_var[i][link][j]['f'][0] - task_var[i][link][j]['r'][0] == task_var[i][link]['L']
                )
            )

100%|██████████| 156/156 [00:54<00:00,  2.84it/s]


\begin{equation}
\begin{array}{r}
\forall i, j, k, l\left|m_{i j}, m_{k l} \in M, \forall x, y\right| v_x, v_y \in V, \\
\left(\varkappa_{i j x}=1 \wedge \varkappa_{k l x}=1\right) \vee\left(\varkappa_{i j x}=0 \wedge \varkappa_{k l x}=0\right): \\
\left(\eta_{i j x}=\eta_{k l x}=v_y\right) \wedge(i \neq k \vee j \neq l) \Longrightarrow \\
r_{i j x}-r_{k l x} \geq \gamma_{k x y}+\zeta_{x y} \vee r_{k l x}-r_{i j x} \geq \gamma_{i x y}+\zeta_{x y} .
\end{array}
\end{equation}

\begin{equation}
\begin{array}{r}
\forall i, j, k, l\left|m_{i j}, m_{k l} \in M, \forall x, y\right| v_x, v_y \in V, \\
\eta_{i j x}=\eta_{k l x}=v_y \wedge \varkappa_{i j k}=1 \wedge \varkappa_{k l k}=0, \\
\forall y \in\{1, \ldots, \lambda-1\}, \forall z \in\{1, \ldots, \lambda-1\} \backslash\{y, y+1\}: \\
\left(f_{i j x}^1 \leq r_{k l x}^1\right) \vee\left(f_{k l x}^1 \leq r_{i j x}^1\right) \\
\vee\left(f_{k l x}^y-r_{k l x}^y \geq \mu\right. \\
\wedge f_{k l x}^{y+1}-r_{k l x}^{y+1} \geq \mu \wedge\left(f_{k l x}^z-r_{k l x}^z=0 \vee f_{k l x}^z-r_{k l x}^z \geq \mu\right) \\
\wedge\left(f_{k l x}^y=r_{i j x}^1 \vee f_{k l x}^y+\mu \leq r_{i j x}^1\right) \\
\wedge\left(r_{k l x}^{y+1}=f_{i j x}^1 \vee r_{k l x}^{y+1} \geq f_{i j x}^1+\mu\right) \\
\left.\wedge f_{i j x}^1-r_{i j x}^1=\gamma_{i x y}+\zeta_{x y}\right) \\
+\left(f_{k l x}^{\lambda-1}-r_{k l x}^{\lambda-1}\right)-\operatorname{If}\left(f_{k l x}^{\lambda-1}-r_{k l x}^{\lambda-1}>0,1,0\right) * \rho_{x y} \\
+\ldots+\left(f_{k l x}^1-r_{k l x}^1\right)-\operatorname{If}\left(f_{k l x}^1-r_{k l x}^1>0,1,0\right) * \rho_{x y} \\
=\gamma_{k x y}^\lambda+\zeta_{x y}-\rho_{x y}
\end{array}
\end{equation}

In [78]:
for i in task_var:
    for link in task_var[i]:
        for j in range(int(LCM / task_var[i][link]['T'])):
            for p in range(0, U - 1):
                s.add(
                    task_var[i][link][j]['r'][p] <= task_var[i][link][j]['f'][p],
                    task_var[i][link][j]['f'][p] <= task_var[i][link][j]['r'][p + 1] ,
                    task_var[i][link][j]['r'][p + 1]  <= task_var[i][link][j]['f'][p + 1]
                )
            s.add(
                z3.Sum([
                    (task_var[i][link][j]['f'][z] - task_var[i][link][j]['r'][z]) 
                    for z in range(0, U)
                    ]) == task_var[i][link]['L']
            )

In [79]:
# for link in tqdm(net_var):
#     for i, j in [(i, j) for i in task_var for j in task_var 
#                  if i != j and link in task_var[i] and link in task_var[j]]:
#         ti, tj = int(task.loc[i].period), int(task.loc[j].period)
#         lcm = np.lcm(ti, tj)
#         x_ijk = task_var[i][link]['x']
#         x_klk = task_var[i][link]['x']
#         for a, b in [(a,b) for a in range(0, int(lcm / ti)) for b in range(0, int(lcm / tj))]:
#             r_ijx, f_ijx = [x + a * ti for x in task_var[i][link]['r']], [x + a * ti for x in task_var[i][link]['f']]
#             r_klx, f_klx = [x + b * tj for x in task_var[j][link]['r']], [x + b * tj for x in task_var[j][link]['f']]
#             for y in range(0, U - 1):
#                 for z in set(range(0, U - 1)) - set([y, y + 1]):
#                     s.add(
#                         z3.Implies(
#                             z3.And(x_ijk == True, x_klk == False)
#                             ,
#                             z3.Or(
#                                 f_ijx[0] <= r_klx[0],
#                                 f_klx[0] <= r_ijx[0],
#                                 z3.And(
#                                     f_klx[y] - r_klx[y] >= 1,
#                                     f_klx[y + 1] - r_klx[y + 1] >= 1,
#                                     z3.Or(
#                                         f_klx[y] - r_klx[z] == 0,
#                                         f_klx[y] - r_klx[z] >= 1
#                                     ),
#                                     z3.Or(
#                                         f_klx[y] == r_ijx[0],
#                                         f_klx[y] + 1 <= r_ijx[0]
#                                     ),
#                                     z3.Or(
#                                         r_klx[y + 1] == f_ijx[0],
#                                         r_klx[y + 1] >= f_ijx[0] + 1
#                                     ),
#                                     f_ijx[0] - r_ijx[0] == task_var[i][link]['L']
#                                 )
#                             )
#                         )
#                     )


\begin{equation}
\begin{array}{r}
\forall i, j\left|m_{i j} \in M, \forall x\right| v_x \in V \\
\forall p \in\{1, \ldots, \lambda-1\}, \varkappa_{i j x}=0 \\
r_{i j x}^p \leq f_{i j x}^p \leq r_{i j x}^{p+1} \leq f_{i j x}^{p+1}
\end{array}
\end{equation}

In [80]:
# for i in task_var:
#     for link in task_var[i]:
#         for p in range(0, U - 1):
#             s.add(
#                     z3.And(
#                         task_var[i][link]['r'][p] <= task_var[i][link]['f'][p],
#                         task_var[i][link]['f'][p] <= task_var[i][link]['r'][p + 1],
#                         task_var[i][link]['r'][p + 1] <= task_var[i][link]['f'][p + 1]
#                     )
#                 )
            

        

In [81]:
# s.add(
#                         z3.Implies(
#                             z3.And(x_ijk == True, x_klk == False)
#                             ,
#                             z3.And(
#                                 z3.Or(
#                                     f_ijx[0] <= r_klx[0],
#                                     f_klx[0] <= r_ijx[0],
#                                     z3.And(
#                                         f_klx[y] - r_klx[y] >= 1,
#                                         f_klx[y + 1] - r_klx[y + 1] >= 1,
#                                         z3.Or(
#                                             f_klx[y] - r_klx[z] == 0,
#                                             f_klx[y] - r_klx[z] >= 1
#                                         ),
#                                         z3.Or(
#                                             f_klx[y] == r_ijx[0],
#                                             f_klx[y] + 1 <= r_ijx[0]
#                                         ),
#                                         z3.Or(
#                                             r_klx[y + 1] == f_ijx[0],
#                                             r_klx[y + 1] >= f_ijx[0] + 1
#                                         ),
#                                         f_ijx[0] - r_ijx[0] == task_var[i][link]['L']
#                                     )
#                                 ),
#                                 z3.Sum(
#                                    [(f_klx[lam] - r_klx[lam]) for lam in range(U)]
#                                 ) == task_var[j][link]['L']
#                             )
#                         )
#                     )

\begin{equation}
\begin{array}{r}
\forall i, j\left|m_{i j} \in M, \forall k, l\right| v_k, v_l \in V: \\
\eta_{i j k}=v_l \Longrightarrow r_{i j l} \geq \max \left(r_{i j k}\right)+\gamma_{i k l}+\zeta_{k l}+s d_l
\end{array}
\end{equation}

In [82]:
for i in task_var.keys():
    path = list(task_var[i].keys())
    for _i, link in enumerate(path[:-1]):
        next_hop = path[_i + 1]
        for j in range(int(LCM / task_var[i][link]['T'])):
            s.add(
                task_var[i][next_hop][j]['r'][0] >= task_var[i][link][j]['f'][U - 1] + net_var[link]['d']
            )

\begin{equation}
\begin{array}{r}
\forall i, j\left|m_{i j} \in M, \forall k\right| v_k \in V, \eta_{i j k}=v_l: \\
\left(\varpi_{k l}=1 \wedge\left(\varkappa_{i j k}=0 \vee \varkappa_{i j k}=1\right)\right) \\
\vee\left(\varpi_{k l}=0 \wedge \varkappa_{i j k}=1\right)
\end{array}
\end{equation}

In [83]:
## We assume every port supports preemption

In [84]:
for i in task_var.keys():
    _hop_s = list(task_var[i].items())[0]
    _hop_e = list(task_var[i].items())[-1]
    for a in range(int(LCM / task_var[i][_hop_s[0]]['T'])):
        s.add(
            _hop_s[1][a]['r'][0] + int(task.loc[i]['deadline']) >=
            _hop_e[1][a]['f'][U - 1] + net_var[_hop_e[0]]['d'] + sync_error
        )

In [85]:
from time import time
start = time()

In [86]:
res = s.check()
result = s.model()

Z3Exception: model is not available

In [ ]:
info = s.statistics()
run_time = info.time
run_memory = info.max_memory

In [ ]:
run_time

369.849

: 

In [ ]:
end = time()
print(end - start)

In [ ]:
## GCL
queue_count = {}
queue_log = {}
GCL = []
for i in task_var:
    for e in task_var[i]:
        queue_count.setdefault(e, 0)
        for j in range(int(LCM / task_var[i][e]['T'])):
            # start = result[task_var[i][e][j]['r'][0]].as_long()
            # end = result[task_var[i][e][j]['f'][-1]].as_long()
            # queue = queue_count[e]
            # GCL.append(
            #         [eval(e), queue, (start) * macrotick, (end) * macrotick, LCM * macrotick]
            #     )
            if eval(str(result[task_var[i][e]['N'][j]])) == True:
                start = result[task_var[i][e][j]['r'][0]].as_long()
                end = result[task_var[i][e][j]['f'][0]].as_long()
                if end > start:
                    queue = queue_count[e]
                    t = task_var[i][e]['T']
                    GCL.append(
                        [eval(e), queue, (start) * macrotick, (end) * macrotick, LCM * macrotick]
                    )
            else:
                for u in range(0, U):
                    start = result[task_var[i][e][j]['r'][u]].as_long()
                    end = result[task_var[i][e][j]['f'][u]].as_long()
                    if end > start:
                        queue = queue_count[e]
                        t = task_var[i][e]['T']
                        GCL.append(
                            [eval(e), queue, (start) * macrotick, (end) * macrotick, LCM * macrotick]
                        )
        queue_log[(i, e)] = queue
        queue_count[e] += 1

In [ ]:
OFFSET = []
for i in task_var:
    e = list(task_var[i].keys())[0]
    for j in range(int(LCM / task_var[i][e]['T'])):
        OFFSET.append(
            [i, 0, (task.loc[i, 'period'] - result[task_var[i][e][j]['r'][0]].as_long()) * macrotick]
        )

In [ ]:
ROUTE = []
for i in task_var:
    route = list(task_var[i].keys())
    for x in route:
        ROUTE.append(
            [i, eval(x)]
        )

QUEUE = []
for i in task_var:
    for e in list(task_var[i].keys()):
        QUEUE.append([i, 0, e, queue_log[(i, e)]])

In [ ]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("ASPDAC2022-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("ASPDAC2022-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("ASPDAC2022-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("ASPDAC2022-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

In [ ]:
for i in task_var:
    _hop_s = list(task_var[i].items())[0]
    _hop_e = list(task_var[i].items())[-1]
    for j in range(int(LCM / task.loc[i, 'period'])):
        print(i, (result[_hop_e[1][j]['f'][-1]].as_long() - result[_hop_s[1][j]['r'][0]].as_long()
         + net_var[_hop_e[0]]['d']) * macrotick)

In [ ]:
task

In [ ]:
# gcl = GCL
# GCL = {}
# CYCLE = {}
# for i, row in gcl.iterrows():
#     GCL.setdefault(row['link'], [])
#     CYCLE.setdefault(row['link'], row['cycle'])
#     GCL[row['link']].append((row['start'], row['end'], row['queue']))
    
# for link in GCL:
#     GCL[link] = sorted(GCL[link], key=lambda x:x[0], reverse=False)
# for link in GCL:
#     temp = GCL[link]
#     for i, row in enumerate(temp[:-1]):
#         if row[1] > temp[i+1][0]:
#             print('overlap', link, row, temp[i+1])